# The building blocks of a ***Decision Tree Classifier***

##### (building discussion around $ WillWait $ example, section 18.3.1 of Artificial Intelligence - A Modern Approach 3rd edition)

In [5]:
using NBInclude                                             # allow code includes to take the form of a notebook
nbinclude( "library.ipynb" );                               # include convenience library functions

---
## Table of Contents

 1. [Set the stage](#Set-the-stage)
 2. [Introducing the Mathematics](#Introducing-the-Mathematics)
 3. Calculate some examples
 

---
## Set the stage 
[TOP](#Table-of-Contents)

The example behind this discussion is the construction of a device that will tell us, given attributes about the restaurant, whether patrons are likely to wait for a table - $ WillWait $. To read the table of data below we consider each column as a variable that can potentially effect that decision. 

The variables in our example have the following meanings:

$ Alt $ : Is there at least one similar *alternative* restatuant nearby? (values **True**, **False**)<br/>
$ Bar $ : Does the restaurant have an inside *bar* area to wait in? (values **True**, **False**)<br />
$ Fri $ (Friday or Saturday) : Is it a *Friday* or Saturday? (values **True**, **False**)<br />
$ Hun $ (Hungry): Are the potential patrons *hungry* when making the decision? (values **True**, **False**)<br />
$ Patrons $ : How many people are currently in the restaurant? (values **None**, **Some**, **Full**)<br />
$ Price $ : How expensive a restaurant is this? (values **\$**, **\$\$**, **\$\$\$**).<br />
$ Raining $ : Is it raining outside? (values **True** or **False**) <br />
$ Reservation $ : Do the potential patrons have a reservation? (values **True** or **False**)<br />
$ Type $: What kind of restaurant is this? (values **French**, **Italian**, **Thai**, **Burger**) <br />
$ WaitEstimate $ : What is the estimated wait time in minutes? (values: **0-10**, **10-30**, **30-60**, or **>60**).

Considering row $ X_7 $ which represents a single instance of training data (and therefore is not a prediction or rule but simply an example).

In [2]:
training_data = readcsv( "training_data.csv" )              # load our training data from local csv file
headers = training_data[1, :]                               # first row are the headers
html_table( training_data[8:8, :], header_row=headers )     # display row 8

Datum,Alt,Bar,Fri,Hun,Patrons,Price,Rain,Res,Type,Est,WillWait
X7,F,T,F,F,None,$,T,F,Burger,0-10,F


We can read this as: 

> for a restaurant with no similar alternatives nearby      ($ Alt $ = **False**)<br />
> that does have an internal bar waiting area               ($ Bar $ = **True**)<br />
> on a Friday or Saturday                                   ($ Fri $ = **True**)<br />
> for hungry potential patrons                              ($ Hun $ = **True**)<br />
> that contains no patrons presently                        ($ Patrons $ = **None**)<br />
> that is only \$ expensive                                 ($ Price $ = **\$**)<br />
> when it is raining                                        ($ Rain $ = **True**)<br />
> when the potential patrons have no reservation            ($ Res $ = **False**)<br />
> the restaurant is a burger joint                          ($ Type $ = **Burger**)<br />
> and the estimated wait is 10 minutes or less              ($ Est $ = **0-10**)<br />
> <br />
> **The potential patrons did not wait**

From Russell and Norwig
> The DECISION-TREE-LEARNING algorithm adopts a greedy divide-and-conquer
> strategy: always test the most important attribute first. This test divides the problem up into
> smaller subproblems that can then be solved recursively. By "most important attribute," we
> mean the one that makes the most difference to the classification of an example.

Finding the most important attribute needs a mathematical definition that we can calculate. The intuition behind our choice is something like "the attribute that helps divide the data most effectively". By dividing data we mean exactly that - for our example we want to ultimately divide the data into instances that results in $ WillWait $ being **True** or **False**. The more categorically an attribute divides input the more "important" that attribute is. 


As one might expect in the world: the attribute $ Patron $ turns out to be a fairly good at helping us determining if potential patrons $ WillWait $. Essentially, for the training data, if there are no patrons $ WillWait $ is **False**, if there are some patrons then $ WillWait $ is **True**. And if the restaurant is already **Full** of $ Patrons $ we need to consider other attributes to determine whether potential patrons will wait or not.  

In contrast, the attribute $ Type $ does not help categorize (or segment or partition) our data. Even in our small number of training data rows a potential patron's choice of restaurant does not seem to impact $ WillWait $ reliably.

We now seek to formalize a method for determining how important an attribute is. The formal way we do this is by examining how well that attribute does at reducing the disorder or *entropy* of the system.

---
## Introducing the Mathematics
[TOP](#Table-of-Contents)

#### How to calculate *entropy*
##### (adapted from (https://bricaud.github.io/personal-blog/entropy-in-decision-trees/)

To help establish how much disorder there is the data we calculate entropy. 

First let us establish the naive probability of an attribute that divides the data into two labels $n,m$:


$$ p(n) = 1 - p(m) $$

since the sum of the probability of n and m equals $ 1 $. Let us say $ q = p(n) = \frac{|n|}{|n|+|m|} $ and $ r = p(m) = \frac{|m|}{|n|+|m|} $ we define their entropy as:

$$ H(m,n) = - q \log(q) - r \log(r) $$

Which generalizes for an attribute that divides data into $ K $ labels:

$$ H = - \sum_{i=1}^K p_i \log(p_i) $$

where $$ p_i = \frac{| \space \text{labelled } K_i \space |}{| \space \text{all labels} \space |} $$  

##### (sourced from section 18.3.4 of Artificial Intelligence - A Modern Approach 3rd edition


In general the entropy for a random variable $ V $ with values $ v_k $ each with a probability $ P(v_k) $, is defined as:

$$ H(V) = \sum_k P(v_k) log_2 \frac{1}{P(v_k)} = - \sum_k P(v_k) log_2 P(v_k) $$

When considering a boolean entropy (a boolean random variable that is true with probability $ q $ we have:

$$ B(q) = - (qlog_2 q) + (1-q)log_2(1-q)) $$

If a training set contains $ p $ positive exampes and $ n $ negative examples, then the entropy of the $ Goal $ attribute on the whole set: 

$$ H(Goal) = B\left(\frac{p}{p+n} \right) $$


For our restaurant example we have 6 positive examples and 6 negative examples

In [ ]:
function H( )

-----
#### Example


For the restaurant data visible below:

There are two labels for WillWait - True or False. When evaluating the entropy for an attribute like $ Patrons $ which can have one of three values **None**, **Some** and **Full** :

 1. $ Patrons $ = **None** <br \>
 "If there are "No" Patrons then $ WillWait $ is always False" gives us $$ $$
 
 
 $$ H \left( \frac{0}{2}, \frac{2}{2} \right) $$
 
 2. Patrons = "Some"
 
 3. Patrons = "Full"
 
 
 "If there are "No" Patrons then $ WillWait $ is always False" gives:
 $$ H\left( \frac{0}{2}, \frac{2}{2} \right) $$
 $$ = |$$ 
 

The probability that $ WillWait = True $ is:




-----

### How to pick nodes
 - A chosen attribute $A$, with $K$ distinct values divides the training set $E$ into subsets $E_1,...E_K$.
 - The __Expected Entropy__ (__EH__) remaining after trying attribute $A$ (with branches $i=1,2,...,K$ is:
 
 $$ EH(A) = \sum_{i=1}^{K} \frac{p_i + n_i}{p + n} \cdot H \left( \frac{p_i}{p_i + n_i},\frac{n_i}{p_i+n_i} \right) $$
 - Where $ p_i $ are the number of datums that do descend along branch $ i $, while $ n_i $ are the number that do not. 
 
 - The __Information Gain__ (__I__) or reduction in entopy for this attribute is:
 
 $$ I(A) = H \left( \frac{p}{p+n},\frac{n}{p+n} \right) - EH(A) $$
 
 - Choose the attribute with the highest I.
 
 <mark>TODO: Understand what the p and q (without subscripts) represent above. Also, watch the Google video on creating a classifier now!</mark>

In [3]:
# #Pkg.add("CSV")

# show more than 80 characters in a row
# ENV["COLUMNS"] = 120 # not needed now we have html_table to format data
training_data = readcsv( "training_data.csv" )

headers = training_data[1, :]
html_table( training_data[2:end, :], header_row=headers, title="Training Data" )


Datum,Alt,Bar,Fri,Hun,Patrons,Price,Rain,Res,Type,Est,WillWait
X1,T,F,F,T,Some,$$$,F,T,French,0-10,T
X2,T,F,F,T,Full,$,F,F,Thai,30-60,F
X3,F,T,F,F,Some,$,F,F,Burger,0-10,T
X4,T,F,T,T,Full,$,F,F,Thai,10-30,T
X5,T,F,T,F,Full,$$$,F,T,French,>60,F
X6,F,T,F,T,Some,$$,F,T,Italian,0-10,T
X7,F,T,F,F,None,$,T,F,Burger,0-10,F
X8,F,F,F,T,Some,$$,T,T,Thai,0-10,T
X9,F,T,T,F,Full,$,T,F,Burger,>60,F
X10,T,T,T,T,Full,$$$,F,T,Italian,10-30,F


For the above data I seek to find the attribute that best splits the data into "WillWait" or not. 

For a training set like this with positive and negative examples we can use the following:

$$ H \left( \frac{p}{p+n}, \frac{n}{p+n} \right) = - \frac{p}{p+n} log_2 \frac{p}{p+n} - \frac{p}{p+n} log_2 \frac{p}{p+n} $$

For this training set, lets consider how _good_ the attributes "Patron" and "Type" are at splitting the "WillWait" values:

For the training set $ p = n = 6 $ (there are six negative "WillWait" values and six positive) = 1 bit

$$ IG(Patrons) = 1 - \left[ EH(Patrons_{None}) + EH(Patrons_{Some}) + EH(Patrons_{Full}) \right] $$

$$ IG(Patrons) = 1 - \left[ \frac{2}{12}H \left( \frac{0}{2}, \frac{2}{2} \right) + \frac{4}{12}H \left( \frac{4}{4}, \frac{0}{4} \right) + \frac{6}{12}H\left( \frac{2}{6}, \frac{4}{6} \right) \right] = 0.0541 bits $$

In [4]:
# allow user to enter attribute and calcular Information Gain (IG) for it

#test_attribute = IJulia.readprompt( "Test attribute" )
#IJulia.clear_output()
#show( test_attribute )